In [1]:
import pandas as pd
from shapely.geometry import Point, shape

from flask import Flask
from flask import render_template
import json



In [28]:
data_path = './input/'
n_samples = 30000

In [46]:
def get_running_slot(hours):
    if 10>hours>=6 :
        return '1'
    elif 14>hours>=10:
        return '2'
    elif 18>hours>=14:
        return '3'
    elif 22>hours>=18:
        return '4'
    elif hours>=22 or hours<2:
        return '5'
    elif 6>hours>=2:
        return '6'
    else: 
        return '0'

In [47]:
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt["date_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[0])
dt["time_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[1].split('-')[0])
dt["timestamp"] = dt["date_of_run"]+' '+dt["time_of_run"]
dt["date_of_joining"] = dt["date_joined"].apply(lambda date: date.split(' ')[0]) +' '+ dt["date_joined"].apply(lambda date: date.split(' ')[1].split('-')[0].split('.')[0])
dt["date_of_joining"]=dt["date_of_joining"].apply(lambda date: date.split(' ')[0])
dt["hr"]=dt["time_of_run"].apply(lambda d: d.split(':')[0]).astype(int)    
dt["running_slot"] = dt["hr"].apply(lambda hours: get_running_slot(hours))
dt["userid"] = dt["user_id"]
cols_to_keep = ['run_id', 'start_time', 'avg_speed', 'distance',
       'run_amount', 'no_of_steps', 'user_id',
       'first_name', 'last_name', 'date_joined','running_slot',
       'cause_id', 'cause_title', 'date_of_run', 'time_of_run',
       'timestamp','userid', 'date_of_joining']
dt = dt.dropna()

In [50]:
d1 = dt[['date_of_run','running_slot','hr','userid','date_of_joining','timestamp']]
d1.head(1)
p = d1[d1['date_of_run']=='2016-06-04']
print len(p)
p

11


,date_of_run,running_slot,hr,userid,date_of_joining,timestamp
7,2016-06-04,6,3,16,2016-05-26,2016-06-04 03:44:40
8,2016-06-04,6,3,16,2016-05-26,2016-06-04 03:48:29
9,2016-06-04,6,3,16,2016-05-26,2016-06-04 03:48:54
10,2016-06-04,6,3,16,2016-05-26,2016-06-04 03:50:09
11,2016-06-04,1,6,16,2016-05-26,2016-06-04 06:14:34
13,2016-06-04,1,8,11,2016-05-21,2016-06-04 08:50:18
14,2016-06-04,1,8,11,2016-05-21,2016-06-04 08:54:34
31,2016-06-04,4,21,23,2016-06-05,2016-06-04 21:08:45
35,2016-06-04,4,21,23,2016-06-05,2016-06-04 21:25:44
44,2016-06-04,1,8,11,2016-05-21,2016-06-04 08:50:18


In [3]:
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')

In [4]:
import datetime
print datetime.date.today()


dt["date_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[0])
dt["time_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[1].split('-')[0])
# df['State'] = df.area.apply(lambda x: x.split(',')[1] if len(x.split(','))>1 else '')

# dt["time_of_run"]=df['phone_brand'].apply(lambda phone_brand: top_10_brands_en[phone_brand] 
#                                                     if (phone_brand in top_10_brands_en) else 'Other')

2016-10-19


In [26]:
dt["timestamp"] = dt["date_of_run"]+' '+dt["time_of_run"]

In [32]:
data_path = './input/'
gen_age_tr = pd.read_csv(data_path + 'gender_age_train.csv')

dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt["date_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[0])
dt["time_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[1].split('-')[0])
dt["timestamp"] = dt["date_of_run"]+' '+dt["time_of_run"]

ev = pd.read_csv(data_path + 'events.csv')
ph_br_dev_model = pd.read_csv(data_path + 'phone_brand_device_model.csv')

df = gen_age_tr.merge(ev, how='left', on='device_id')
df = df.merge(ph_br_dev_model, how='left', on='device_id')

    #Get n_samples records

In [35]:
df = df.merge(dt, how='left', on='timestamp')
df.head()
gen_age_tr.head()

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [11]:
dt_clean = dt["date_of_run"]

dt_clean.to_json(orient='records')
["2","2","1","0","3","3","3","3"]
cols_to_keeps = ['date_of_run', 'time_of_run']
dt_clean = dt[cols_to_keeps].dropna()
records = dt_clean.to_json(orient='records')

In [17]:
def get_age_segment(age):
    if age <= 22:
        return '22-'
    elif age <= 26:
        return '23-26'
    elif age <= 28:
        return '27-28'
    elif age <= 32:
        return '29-32'
    elif age <= 38:
        return '33-38'
    else:
        return '39+'

def get_location(longitude, latitude, provinces_json):
    
    point = Point(longitude, latitude)

    for record in provinces_json['features']:
        polygon = shape(record['geometry'])
        if polygon.contains(point):
            return record['properties']['name']
    return 'other'

data_path = './input/'
with open(data_path + '/geojson/china_provinces_en.json') as data_file:    
    provinces_json = json.load(data_file)
    
data_path = './input/dataaa/'
gen_age_tr = pd.read_csv(data_path + 'gender_age_train.csv')
ev = pd.read_csv(data_path + 'events.csv')
ph_br_dev_model = pd.read_csv(data_path + 'phone_brand_device_model.csv')

df = gen_age_tr.merge(ev, how='left', on='device_id')
df = df.merge(ph_br_dev_model, how='left', on='device_id')
    #Get n_samples records
df = df[df['longitude'] != 0].sample(n=n_samples)


top_10_brands_en = {'华为':'Huawei', '小米':'Xiaomi', '三星':'Samsung', 'vivo':'vivo', 'OPPO':'OPPO',
                        '魅族':'Meizu', '酷派':'Coolpad', '乐视':'LeEco', '联想':'Lenovo', 'HTC':'HTC'}

df['phone_brand_en'] = df['phone_brand'].apply(lambda phone_brand: top_10_brands_en[phone_brand] 
                                                    if (phone_brand in top_10_brands_en) else 'Other')

df['age_segment'] = df['age'].apply(lambda age: get_age_segment(age))

df['location'] = df.apply(lambda row: get_location(row['longitude'], row['latitude'], provinces_json), axis=1)

cols_to_keep = ['timestamp', 'longitude', 'latitude', 'phone_brand_en', 'gender', 'age_segment', 'location']
df_clean = df[cols_to_keep].dropna()


In [18]:
re = df_clean.to_json(orient='records')

In [22]:
# '[{"timestamp":"2016-05-06 15:40:12","longitude":124.02,"latitude":45.0,"phone_brand_en":"OPPO","gender":"F","age_segment":"22-","location":"Jilin"},{"timestamp":"2016-05-05 21:47:04","longitude":115.26,"latitude":33.05,"phone_brand_en":"vivo","gender":"M","age_segment":"27-28","location":"Anhui"},
# '[{"date_of_run":"2016-07-29","time_of_run":"07:01:37"},{"date_of_run":"2016-07-31","time_of_run":"07:56:05"}
#records

In [80]:
import pandas as pd
data_path = './input/'
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt.head()

,run_id,start_time,avg_speed,distance,user_id_id,run_amount,run_duration,no_of_steps,user_id,first_name,last_name,gender_user,date_joined,cause_id,cause_title
0,1284,2016-07-29 07:01:37-04,1.483692,1.985234,81,20,00:23:00,61001,81,Manoj,Khandelwal,m,2016-07-08 03:30:30.476094-04,4,For Our Farmers
1,1302,2016-07-31 07:56:05-04,0.524181,2.371040,191,25,01:16:11,184565,191,suyesh,dolas,male,2016-07-31 06:16:27.847882-04,4,For Our Farmers
2,1319,2016-08-01 01:44:41-04,0.037516,5.402810,36,54,4 days 13:17:34,33010,36,Prashant,Gudibanda,m,2016-06-13 08:29:29.517684-04,6,Defeating Cancer Together
3,1335,2016-07-28 05:45:51-04,1.035493,2.729146,104,28,00:44:00,190329,104,Shubham,Saraogi,m,2016-07-15 05:07:15.496121-04,7,Uttarkashi School
4,1350,2016-08-02 06:22:10-04,6.097581,0.118226,184,3,00:00:44,0,184,Ritesh,Soni,m,2016-07-31 05:45:55.031873-04,6,Defeating Cancer Together


In [37]:
import pandas as pd
data_path = './input/'
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt["date_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[0])
dt["time_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[1].split('-')[0])
dt["timestamp"] = dt["date_of_run"]+' '+dt["time_of_run"]
dt["date_of_joining"] = dt["date_joined"].apply(lambda date: date.split(' ')[0])
dt['runners'] = dt.groupby('date_of_run')['date_of_run'].transform('count')
dh = dt.groupby(['date_of_run']).apply(lambda s:len(s['user_id'].unique()))
f = pd.DataFrame({'date_of_run':dh.index, 'uniq_runners':dh.values})
dt = dt.merge(f, how='left', on='date_of_run')
dt["userid"] = dt["user_id"]

In [42]:
import numpy as np
dh = dt.groupby(['date_of_run']).apply(lambda s:s['user_id'].unique())
f = pd.DataFrame({'date_of_run':dh.index, 'uniq_runners':dh.values})
f
# dh = np.unique(dt.groupby('date_of_run')['user_id'])
dh = dt[["date_of_run","user_id"]]
d =dh.drop_duplicates
fg = dt.groupby(["date_of_run","user_id"]).size().reset_index().rename(columns={0:'count'})


In [60]:
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt["date_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[0])
dt["time_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[1].split('-')[0])
dt["timestamp"] = dt["date_of_run"]+' '+dt["time_of_run"]    
dw = dt.groupby(["date_of_run","user_id"]).size().reset_index().rename(columns={0:'count'})

In [84]:
import pandas as pd
data_path = './input/'
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt["date_of_joining"] = dt["date_joined"].apply(lambda date: date.split(' ')[0])
d =dt.drop_duplicates(["date_of_joining","user_id"])
d.head()


,run_id,start_time,avg_speed,distance,user_id_id,run_amount,run_duration,no_of_steps,user_id,first_name,last_name,gender_user,date_joined,cause_id,cause_title,date_of_joining
0,1284,2016-07-29 07:01:37-04,1.483692,1.985234,81,20,00:23:00,61001,81,Manoj,Khandelwal,m,2016-07-08 03:30:30.476094-04,4,For Our Farmers,2016-07-08
1,1302,2016-07-31 07:56:05-04,0.524181,2.371040,191,25,01:16:11,184565,191,suyesh,dolas,male,2016-07-31 06:16:27.847882-04,4,For Our Farmers,2016-07-31
2,1319,2016-08-01 01:44:41-04,0.037516,5.402810,36,54,4 days 13:17:34,33010,36,Prashant,Gudibanda,m,2016-06-13 08:29:29.517684-04,6,Defeating Cancer Together,2016-06-13
3,1335,2016-07-28 05:45:51-04,1.035493,2.729146,104,28,00:44:00,190329,104,Shubham,Saraogi,m,2016-07-15 05:07:15.496121-04,7,Uttarkashi School,2016-07-15
4,1350,2016-08-02 06:22:10-04,6.097581,0.118226,184,3,00:00:44,0,184,Ritesh,Soni,m,2016-07-31 05:45:55.031873-04,6,Defeating Cancer Together,2016-07-31


In [64]:
df = dt.merge(dw, how='left', on="date_of_run")

In [65]:
len(dt.dropna())

2315

In [69]:
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt["date_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[0])
dt["time_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[1].split('-')[0])
dt["timestamp"] = dt["date_of_run"]+' '+dt["time_of_run"]    
 
cols_to_keep = ['timestamp','user_id']
dt_clean = dt[cols_to_keep].dropna()
len(dt_clean)

1537

In [81]:
dt_clean.head()

,timestamp,user_id
0,2016-07-29 07:01:37,81
1,2016-07-31 07:56:05,191
2,2016-08-01 01:44:41,36
3,2016-07-28 05:45:51,104
4,2016-08-02 06:22:10,184


In [78]:
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt["date_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[0])
dt["time_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[1].split('-')[0])
dt["timestamp"] = dt["date_of_run"]+' '+dt["time_of_run"] 
for i in xrange(0,len(dt["user_id"])):
    dt["user_id"][i]

In [8]:
def get_running_slot(hours):
    if 10>hours>=6 :
        return '6-10'
    elif 14>hours>=10:
        return '10-14'
    elif 18>hours>=14:
        return '14-18'
    elif 22>hours>=18:
        return '18-22'
    elif hours>22 or hours<2:
        return '22-2'
    elif 6>hours>=2:
        return '2-6'
    else: 
        return 'NAN'

data_path = './input/'
import pandas as pd
dt = pd.read_csv(data_path + 'sample_runs_11102016.csv')
dt["date_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[0])
dt["time_of_run"]=dt["start_time"].apply(lambda date: date.split(' ')[1].split('-')[0])
dt["timestamp"] = dt["date_of_run"]+' '+dt["time_of_run"]
dt["date_of_joining"] = dt["date_joined"].apply(lambda date: date.split(' ')[0]) +' '+ dt["date_joined"].apply(lambda date: date.split(' ')[1].split('-')[0].split('.')[0])

dt["hr"]=dt["time_of_run"].apply(lambda d: d.split(':')[0]).astype(int)    
dt["running_slot"] = dt["hr"].apply(lambda hours: get_running_slot(hours))
    #df['age_segment'] = df['age'].apply(lambda age: get_age_segment(age))

    #dt["date_of_joining"] = dt["date_joined"].apply(lambda date: date.split(' ')[0])     
dt['runners'] = dt.groupby('date_of_run')['date_of_run'].transform('count')
df = dt.groupby(['date_of_run']).apply(lambda s:len(s['user_id'].unique()))
f = pd.DataFrame({'date_of_run':df.index, 'uniq_runners':df.values})
dt = dt.merge(f, how='left', on='date_of_run')
dt["userid"] = dt["user_id"]
cols_to_keep = ['timestamp','running_slot']
dt[cols_to_keep]
dt[]

,timestamp,running_slot
0,2016-07-29 07:01:37,6-10
1,2016-07-31 07:56:05,6-10
2,2016-08-01 01:44:41,22-2
3,2016-07-28 05:45:51,2-6
4,2016-08-02 06:22:10,6-10
5,2016-07-28 05:05:10,2-6
6,2016-08-05 06:39:19,6-10
7,2016-06-04 03:44:40,2-6
8,2016-06-04 03:48:29,2-6
9,2016-06-04 03:48:54,2-6
